# Extra practices

The extra pratices I will do in a seperate notebook

These are the things I practiced:

* heavy storms in The Netherlands, <a href="https://www.knmi.nl/nederland-nu/klimatologie/lijsten/zwarestormen">website</a>. So, it is possible in one go, without BeautifulSoup!
* storm surge floods in The Netherlands (stormvloed)


In [1]:
import pandas as pd
storms_NL = pd.read_html('https://www.knmi.nl/nederland-nu/klimatologie/lijsten/zwarestormen')

In [2]:
storms_NL[0]

,Nr.,Jaar,Datum,Hoogste windsneldheid Bft,Hoogste uur m/s,Hoogste uur km/h,Zwaarste windstoot m/s,Zwaarste windstoot km/h,Plaats,Naam
0,1,1911,30 sep/ 1 okt,11,30,108,38.0,137.0,Hoek van Holland,NaN
1,2,1912,27 aug,10,27,97,41.0,148.0,Hoek van Holland,NaN
2,3,1913,26/27 dec,10,28,101,NaN,NaN,Vlissingen,NaN
3,4,1914,12 aug,10,26,94,NaN,NaN,Den Helder,NaN
4,5,1914,11 nov,10,26,94,NaN,NaN,Vlissingen,NaN
...,...,...,...,...,...,...,...,...,...,...
61,62,2016,20 nov,10,26,94,37.0,133.0,IJmuiden,NaN
62,63,2017,13 sep,10,26,94,35.0,126.0,Vlieland,NaN
63,64,2018,3 jan,10,26,94,34.0,122.0,Vlieland,NaN
64,65,2018,18 jan,11,30,108,40.0,144.0,Hoek van Holland,NaN


## Stormvloed in Nederland

I wanted to try this one, because it's one of the things I am really interested in. It is about all heavy storms in Dutch history. This kind of storm is called Stormvloed in Dutch, which referes to high tide and heavy storm, which always make Dutch people a bit nervous. Stormvloed is mentioned <a href="https://nl.wikipedia.org/wiki/Overstroming">here</a> and I want to scrape the data from the individual pages. I only try the list from the sea. 

In [3]:
from bs4 import BeautifulSoup
import requests
from random import randint  
from time import sleep     

In [4]:
url = "https://nl.wikipedia.org/wiki/Overstroming"
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [5]:
storm_soup = BeautifulSoup(response.content, "html.parser")

In [6]:
# 2nd is the first storm
storm_soup.select("div.mw-parser-output > p:nth-child(16) > a:nth-child(2)")

[<a href="/wiki/Stormvloed_van_838" title="Stormvloed van 838">838</a>]

In [7]:
# 52nd is the last storm
storm_soup.select("div.mw-parser-output > p:nth-child(16) > a:nth-child(52)")

[<a href="/wiki/Allerheiligenvloed_(2006)" title="Allerheiligenvloed (2006)">2006</a>]

In [8]:
# Constructing the url list with all the sea storms for the main loop
domain = "https://nl.wikipedia.org"
wiki = []
storm = []

for i in range(2,53):
    wiki.append(i)

for w in wiki:
    if len(storm_soup.select("div.mw-parser-output > p:nth-child(16) > a:nth-child(" + str(w) + ")")) == 0:
        # span exception in the year 1288
        storm = storm + storm_soup.select("#mw-content-text > div.mw-parser-output > p:nth-child(16) > span:nth-child(" + str(w) + ") > a")
    else:
        # normal 
        storm = storm + storm_soup.select("div.mw-parser-output > p:nth-child(16) > a:nth-child(" + str(w) + ")")


In [9]:
#storm

In [10]:
ctr = 0
storm_name = []
storm_year = []
storm_url = []
year = []
date = []
region = []
death_toll = []
soup_ibox = []

for i in storm:
    wait_time = randint(1,4)
    sleep(wait_time)
    url = domain + storm[ctr]["href"]
    storm_name.append(storm[ctr]["title"])
    storm_year.append(storm[ctr].get_text())
    storm_url.append(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    # table infobox on each sub page contains all information
    # so create soup of subsets: all infoboxes
    soup_ibox.append(soup.select("table.infobox"))
    # Year
    try:
        year.append(soup_ibox[ctr][0].find("td", string = "Jaar").next_sibling.next_sibling.get_text().replace('\n',''))
    except:
        year.append("nb")
    # Date
    try:
        date.append(soup_ibox[ctr][0].find("td", string = "Datum").next_sibling.next_sibling.get_text().replace('\n',''))
    except:
        date.append("nb")
    # Region
    try:
        region.append(soup_ibox[ctr][0].find("td", string = "Regio").next_sibling.next_sibling.get_text().replace('\n',''))
    except:
        region.append("nb") 
    # Death toll
    try:
        death_toll.append(soup_ibox[ctr][0].find("td", string = "Doden").next_sibling.next_sibling.get_text().replace('\n',''))
    except:
        death_toll.append("nb")
    print(ctr, "processed:", url, "with wait time:", wait_time, response.status_code)
    ctr += 1


0 processed: https://nl.wikipedia.org/wiki/Stormvloed_van_838 with wait time: 3 200
1 processed: https://nl.wikipedia.org/wiki/Stormvloed_van_1014 with wait time: 4 200
2 processed: https://nl.wikipedia.org/wiki/Stormvloed_van_1042 with wait time: 2 200
3 processed: https://nl.wikipedia.org/wiki/Stormvloed_van_1134 with wait time: 4 200
4 processed: https://nl.wikipedia.org/wiki/Sint-Thomasvloed with wait time: 2 200
5 processed: https://nl.wikipedia.org/wiki/Sint-Julianavloed with wait time: 1 200
6 processed: https://nl.wikipedia.org/wiki/Allerheiligenvloed_(1170) with wait time: 2 200
7 processed: https://nl.wikipedia.org/wiki/Sint-Nicolaasvloed with wait time: 2 200
8 processed: https://nl.wikipedia.org/wiki/Stormvloed_van_1212 with wait time: 4 200
9 processed: https://nl.wikipedia.org/wiki/Stormvloed_van_1214 with wait time: 1 200
10 processed: https://nl.wikipedia.org/wiki/Sint-Marcellusvloed_(1219) with wait time: 4 200
11 processed: https://nl.wikipedia.org/wiki/Stormvloed_van

In [11]:
stormvloed = pd.DataFrame({"Year":storm_year,
                           "Storm name":storm_name,
                           "url":storm_url,
                           "Storm year":year,
                           "Date":date,
                           "Region":region,
                           "Death toll":death_toll
                          })
stormvloed

,Year,Storm name,url,Storm year,Date,Region,Death toll
0,838,Stormvloed van 838,https://nl.wikipedia.org/wiki/Stormvloed_van_838,838,26 december,Noordwest-Nederland,ruim 2.400
1,1014,Stormvloed van 1014,https://nl.wikipedia.org/wiki/Stormvloed_van_1014,nb,28 op 29 september 1014,"Engeland, Zuidwest Nederland en Vlaanderen",Duizenden
2,1042,Stormvloed van 1042,https://nl.wikipedia.org/wiki/Stormvloed_van_1042,nb,2 november 1042,Vlaanderen,nb
3,1134,Stormvloed van 1134,https://nl.wikipedia.org/wiki/Stormvloed_van_1134,nb,4 oktober 1134,Zuidwest-Nederland en Vlaanderen,nb
4,1163,Sint-Thomasvloed,https://nl.wikipedia.org/wiki/Sint-Thomasvloed,1163,21 december,Midden-Nederland,nb
5,1164,Sint-Julianavloed,https://nl.wikipedia.org/wiki/Sint-Julianavloed,nb,16 op 17 februari 1164,Noord Nederland en Duitsland,Duizenden
6,1170,Allerheiligenvloed (1170),https://nl.wikipedia.org/wiki/Allerheiligenvlo...,nb,1 - 2 november 1170,Noordwest-Nederland,nb
7,1196,Sint-Nicolaasvloed,https://nl.wikipedia.org/wiki/Sint-Nicolaasvloed,nb,december 1196,Noord-west Nederland,nb
8,1212,Stormvloed van 1212,https://nl.wikipedia.org/wiki/Stormvloed_van_1212,1212,nb,Noord Nederland,36.000/60.000 (beide twijfelachtig)
9,1214,Stormvloed van 1214,https://nl.wikipedia.org/wiki/Stormvloed_van_1214,1214,nb,Nederland,nb


In [ ]:
# So, it's better to learn how to swim in NL. And the year on the main page contains better info
# than the year in the table/infobox of the subpages